In [1]:
# Подключаем библиотеки
import os
os.chdir("..")

from compiled_wrapper import scidi_wrapper

import pandas
import numpy as np
import json
import random
import time 
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# Загружаем данные с одинарным шумом
data = pandas.read_csv('test_data/numbers/digits+noise.csv', header=None)
data = data[data.columns[:-1]]
# Преобразуем в строку (оболочка работает пока только со строковым представлением)
str_data = data.values.astype(str)
print str_data[:5]

[['6' '2' '2' '7' '5' '1' '1' '3' '5' '1' '1' '3' '5' '1' '1' '3' '5' '8'
  '1' '3' '7' '4' '4' '6']
 ['6' '2' '2' '7' '5' '1' '1' '8' '5' '1' '1' '3' '5' '1' '1' '3' '5' '1'
  '1' '3' '7' '4' '4' '6']
 ['6' '2' '2' '7' '5' '1' '1' '3' '5' '1' '1' '3' '5' '1' '1' '3' '5' '1'
  '1' '3' '7' '8' '4' '6']
 ['6' '2' '2' '7' '5' '1' '1' '3' '8' '1' '1' '3' '5' '1' '1' '3' '5' '1'
  '1' '3' '7' '4' '4' '6']
 ['6' '2' '8' '7' '5' '1' '1' '3' '5' '1' '1' '3' '5' '1' '1' '3' '5' '1'
  '1' '3' '7' '4' '4' '6']]


In [3]:
len(str_data[0])

24

In [4]:
191 / 8

23

In this task we need rules with negetion. There are no negation in sd library so we need to simulate it with dummy predicates.

In [5]:
# Amount of dummy cols we need:
print len(str_data[0]) * 8

192


In [6]:
# function for dummy negation
def makeDummyEncoding(item):
    new_item = []
    for i in range(len(item)):
        for j in range(1, 9):
            if item[i] != str(j):
                new_item.append('0')
            else:
                new_item.append('1')
    return new_item

In [7]:
# Напишем функцию, что правила-отрицания приведет в нормальный вид:
test_rule = "59=0 & 69=0 => 37=0"

def normalizeDummyEncoding(rule):
    preds = rule.split()
    for i, pred in enumerate(preds):
        if '=' in pred and not '=>' in pred:
            code, val = pred.split('=')
            code = int(code)
            shift = (code - 1) / 8 + 1
            level = (code - 1) % 8 + 1
            if val == '1':
                preds[i] = str(shift) + '=' + str(level)
            else:
                preds[i] = str(shift) + '=' + ' not ' + str(level)
                
    return " ".join(preds)

normalizeDummyEncoding(test_rule)

'8= not 3 & 9= not 5 => 5= not 5'

In [8]:
# Let's generate dummy negation data
data_with_negations = np.array([makeDummyEncoding(x) for x in str_data])

In [9]:
# Now we have 216 attributes! That is bad 
data_with_negations.shape

(360, 192)

In [10]:
# Nevertheless, let's generate rules!
sw = scidi_wrapper.ScidiWrapper()

In [11]:
data_with_negations[0]

array(['0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0',
       '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0',
       '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0

In [ ]:
%%time
sw.setData(data_with_negations[:])
sw.makeRulesWithSDGenerator(2, 0.6, 1, 0.95)
d2_rules = sw.getRules()
print len(d2_rules)
print max([len(r) for r in d2_rules])

In [ ]:
m_rules = [normalizeDummyEncoding(rule) for rule in d2_rules]

In [ ]:
lens = np.array([len(r) for r in m_rules])

In [ ]:
f = open('rules_d2.txt', 'w')
for item in m_rules:
    f.write("%s\n" % item)

In [ ]:
%%time
sw.setData(data_with_negations[:])
sw.makeRulesWithSDGenerator(3, 0.6, 1, 0.95)
d2_rules = sw.getRules()
print len(d2_rules)
print max([len(r) for r in d2_rules])

In [ ]:
m_rules = [normalizeDummyEncoding(rule) for rule in d2_rules]
f = open('rules_d3.txt', 'w')
for item in m_rules:
    f.write("%s\n" % item)